In [1]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn import metrics

In [2]:
cong_data = pd.read_csv("data-1.csv")
cong_data['Traded'] = pd.to_datetime(cong_data['Traded'])
cong_data['Traded'] = cong_data['Traded'].apply(lambda dt: dt.replace(day=1))

In [3]:
cong_data['Published'] = pd.to_datetime(cong_data['Published'])
cong_data['date'] = cong_data['Published'].apply(lambda dt: dt.replace(day=1))

In [4]:
cong_data = cong_data[cong_data['Published'] <= "2023-12-31"]

In [5]:
ret = pd.read_csv("CongressReturns.csv")

In [6]:
ret['date'] = pd.to_datetime(ret['date'])
ret['date'] = ret['date'].apply(lambda dt: dt.replace(day=1))

In [7]:
ret

,PERMNO,date,TICKER,COMNAM,RET
0,10026,2021-01-01,JJSF,J & J SNACK FOODS CORP,-0.017442
1,10026,2021-02-01,JJSF,J & J SNACK FOODS CORP,0.039958
2,10026,2021-03-01,JJSF,J & J SNACK FOODS CORP,-0.007275
3,10026,2021-04-01,JJSF,J & J SNACK FOODS CORP,0.048271
4,10026,2021-05-01,JJSF,J & J SNACK FOODS CORP,0.066642
...,...,...,...,...,...
336911,93436,2023-08-01,TSLA,TESLA INC,-0.034962
336912,93436,2023-09-01,TSLA,TESLA INC,-0.030456
336913,93436,2023-10-01,TSLA,TESLA INC,-0.197346
336914,93436,2023-11-01,TSLA,TESLA INC,0.195379


In [8]:
cong_data["COMNAM"] = cong_data["Traded Issuer"].apply(lambda st: st.upper().replace(".", " "))
cong_data[cong_data["COMNAM"]=="NVIDIA CORPORATION"]
cong_data["COMNAM"] = cong_data["COMNAM"].apply(lambda x: x.replace("NVIDIA CORPORATION","NVIDIA CORP") if "NVIDIA" in x else x)

In [9]:
int_df = pd.merge(cong_data, ret,how='inner', on=['COMNAM', 'date'])


In [10]:
int_df = int_df.dropna(subset=['RET'])
int_df['Year'] = int_df['date'].dt.year
int_df['Month'] = int_df['date'].dt.month


In [11]:
int_df

,Unnamed: 0,Politician,Traded Issuer,Published,Traded,Filed after,Owner,Type,Size,Price,date,COMNAM,PERMNO,TICKER,RET,Year,Month
0,1745,Markwayne Mullin,VERALTO CORP,2023-12-29,2023-12-01,23,Joint,sell,1K–15K,75.55,2023-12-01,VERALTO CORP,24174,VLTO,0.066019,2023,12
1,2380,Ro Khanna,VERALTO CORP,2023-12-08,2023-11-01,33,Child,sell,1K–15K,70.8,2023-12-01,VERALTO CORP,24174,VLTO,0.066019,2023,12
2,2426,Sheldon Whitehouse,VERALTO CORP,2023-12-08,2023-11-01,8,Spouse,sell,1K–15K,74.22,2023-12-01,VERALTO CORP,24174,VLTO,0.066019,2023,12
3,2427,Sheldon Whitehouse,VERALTO CORP,2023-12-08,2023-11-01,8,Self,sell,1K–15K,74.22,2023-12-01,VERALTO CORP,24174,VLTO,0.066019,2023,12
4,2429,Sheldon Whitehouse,VERALTO CORP,2023-12-08,2023-11-01,8,Self,sell,1K–15K,74.22,2023-12-01,VERALTO CORP,24174,VLTO,0.066019,2023,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20061,37928,Ashley Hinson,Gray Television Inc,2021-04-07,2021-03-01,5,Spouse,sell,1K–15K,18.4,2021-04-01,GRAY TELEVISION INC,39731,GTN,0.071223,2021,4
20062,37928,Ashley Hinson,Gray Television Inc,2021-04-07,2021-03-01,5,Spouse,sell,1K–15K,18.4,2021-04-01,GRAY TELEVISION INC,83989,GTN,0.104348,2021,4
20063,37929,Ashley Hinson,Heron Therapeutics Inc,2021-04-07,2021-03-01,5,Spouse,sell,1K–15K,16.21,2021-04-01,HERON THERAPEUTICS INC,11636,HRTX,0.078347,2021,4
20064,37932,Ashley Hinson,Shake Shack Inc,2021-04-07,2021-03-01,5,Spouse,sell,1K–15K,112.77,2021-04-01,SHAKE SHACK INC,15167,SHAK,-0.035648,2021,4


In [12]:
df_buys = int_df.groupby(["date", "PERMNO"])["Type"].value_counts()

In [13]:
df_buys = pd.DataFrame(df_buys)
dict = {"date":[], "PERMNO": [], "Type": [], "count": []}
for index, row in df_buys.iterrows():
    dict["date"].append(index[0])
    dict["PERMNO"].append(index[1])
    dict["Type"].append(index[2])
    dict["count"].append(row[0])



/var/folders/sz/ts8tdp9n3f133bj7r6__4w7h0000gn/T/ipykernel_46634/382158420.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  dict["count"].append(row[0])


In [14]:
df_all = pd.DataFrame(dict)

In [15]:
df_buys = df_all[df_all["Type"]=="buy"]
df_buys = df_buys.rename(columns={"count":"buy"})

In [16]:
df_sells = df_all[df_all["Type"]=="sell"]
df_sells = df_sells.rename(columns={"count":"sell"})

In [17]:
df_big = pd.concat([df_buys, df_sells])
df_big = df_big.rename(columns={"PERMNO": "permno"})

In [18]:
int_df[int_df["TICKER"] == "AAPL"]

,Unnamed: 0,Politician,Traded Issuer,Published,Traded,Filed after,Owner,Type,Size,Price,date,COMNAM,PERMNO,TICKER,RET,Year,Month
119,2022,John Curtis,Apple Inc,2023-12-18,2023-12-01,11,Joint,sell,1K–15K,189.43,2023-12-01,APPLE INC,14593,AAPL,0.013583,2023,12
722,2897,Josh Gottheimer,Apple Inc,2023-11-13,2023-10-01,12,Joint,buy,1K–15K,170.29,2023-11-01,APPLE INC,14593,AAPL,0.113720,2023,11
723,3059,Ro Khanna,Apple Inc,2023-11-10,2023-10-01,37,Child,buy,1K–15K,173.75,2023-11-01,APPLE INC,14593,AAPL,0.113720,2023,11
724,3403,Ro Khanna,Apple Inc,2023-11-10,2023-10-01,29,Spouse,sell,15K–50K,178.39,2023-11-01,APPLE INC,14593,AAPL,0.113720,2023,11
725,3633,Greg Stanton,Apple Inc,2023-11-09,2023-10-01,16,Undisclosed,sell,15K–50K,173.44,2023-11-01,APPLE INC,14593,AAPL,0.113720,2023,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19394,36660,Pat Fallon,Apple Inc,2021-06-19,2021-04-01,59,Undisclosed,sell,500K–1M,134.84,2021-06-01,APPLE INC,14593,AAPL,0.099109,2021,6
19395,36661,Pat Fallon,Apple Inc,2021-06-19,2021-04-01,59,Undisclosed,sell,250K–500K,134.84,2021-06-01,APPLE INC,14593,AAPL,0.099109,2021,6
19396,37009,Marjorie Taylor Greene,Apple Inc,2021-06-13,2021-05-01,30,Joint,buy,15K–50K,122.77,2021-06-01,APPLE INC,14593,AAPL,0.099109,2021,6
19397,37017,John Boozman,Apple Inc,2021-06-12,2021-05-01,38,Undisclosed,sell,1K–15K,132.54,2021-06-01,APPLE INC,14593,AAPL,0.099109,2021,6


In [19]:
#let's make a new XGB model here using 2021 and 2022 data plus cong trading data
def to_intersect(file, df_returns):
    df = pd.read_csv(file)
    df['yyyymm'] = pd.to_datetime(df['yyyymm'],format = "%Y%m")
    df['yyyymm'] = df['yyyymm'].apply(lambda dt: dt.replace(day=1))
    dfRET = df_returns[df_returns["date"] >= df['yyyymm'].min()]
    dfRET =  dfRET[df_returns["date"]<=df['yyyymm'].max()]
    dfRET.rename(columns={"PERMNO":"permno"}, inplace=True)
    #df.drop(df.columns[len(df.columns)-1], axis=1, inplace=True)
    df.rename(columns={'yyyymm': 'date'}, inplace=True)
    intersection = pd.merge(df, dfRET, how='inner', on=['permno', 'date'])
    intersection['Year'] = intersection['date'].dt.year
    intersection['Month'] = intersection['date'].dt.month
    intersection['RET'] = pd.to_numeric(intersection["RET"], errors='coerce')
    intersection['Quintile'] = intersection.groupby(['Year', 'Month'])['RET'].transform(
        lambda x: pd.qcut(x, 5, labels=False)
    )
    intersection = intersection.sort_values(by=["permno", "date"])
    intersection["New Y"] = intersection.groupby(["permno"])["Quintile"].shift(-1)
    intersection.dropna(subset = ['New Y'], inplace=True)
    intersection["New RET"] = intersection.groupby(["permno"])["RET"].shift(-1)
    intersection.dropna(subset = ["New RET"], inplace=True)
    return intersection

In [54]:
def to_X_Y(intersect):
    intersect.dropna(subset = ['New Y'], inplace=True)
    drop_from_df = ["permno", "date", "RET", "Quintile", "Year", "Month", "Unnamed: 0", "New Y", "New RET", "COMNAM", "TICKER", "Type"]
    Y = intersect["New Y"]
    X = intersect.drop(columns=drop_from_df)
    return [X, Y]

In [21]:
data_21 = to_intersect("2021Test.csv", ret)
data_22 = to_intersect("2022Test.csv", ret)

/var/folders/sz/ts8tdp9n3f133bj7r6__4w7h0000gn/T/ipykernel_46634/2689625411.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dfRET =  dfRET[df_returns["date"]<=df['yyyymm'].max()]


In [49]:
train_data = pd.merge(pd.concat([data_21, data_22]), df_big, how="left", on=["date", "permno"])
train_alt = pd.merge(data_21, df_big, how="inner", on=["date", "permno"])

In [55]:
train_X, train_Y = to_X_Y(train_data)
alt_X, alt_Y = to_X_Y(train_alt)

In [56]:
alt_X

,AM,AOP,AbnormalAccruals,Accruals,AccrualsBM,Activism1,Activism2,AdExp,AgeIPO,AnalystRevision,...,sfe,sinAlgo,skew1,std_turn,tang,zerotrade,zerotradeAlt1,zerotradeAlt12,buy,sell
0,0.810459,-0.055091,-0.024817,0.028172,NaN,NaN,NaN,NaN,NaN,1.102250,...,NaN,NaN,-0.253819,NaN,0.764948,2.126051e-08,2.153034e-08,1.860687e-08,2.0,NaN
1,0.876068,-0.354010,-0.024817,0.028172,NaN,NaN,NaN,NaN,NaN,0.951762,...,NaN,NaN,-0.141269,NaN,0.764948,2.137384e-08,2.165851e-08,1.881996e-08,2.0,NaN
2,0.531166,NaN,-0.064216,0.022860,NaN,NaN,NaN,0.000819,NaN,1.042601,...,NaN,NaN,-0.036734,NaN,NaN,2.638811e-08,2.868774e-08,2.559077e-08,2.0,NaN
3,0.463884,NaN,-0.064216,0.022860,NaN,NaN,NaN,0.000715,NaN,0.997831,...,NaN,NaN,-0.026089,NaN,NaN,2.375697e-08,2.091762e-08,2.565470e-08,1.0,NaN
4,0.463884,NaN,-0.064216,0.022860,NaN,NaN,NaN,0.000715,NaN,0.997831,...,NaN,NaN,-0.026089,NaN,NaN,2.375697e-08,2.091762e-08,2.565470e-08,NaN,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1821,0.070755,-8.036203,0.064442,0.088877,NaN,NaN,NaN,NaN,NaN,1.171111,...,NaN,NaN,-0.066116,NaN,0.752188,3.629791e-09,4.593460e-09,2.733983e-09,NaN,3.0
1822,0.066978,-8.036203,0.064442,0.088877,NaN,NaN,NaN,NaN,NaN,1.005693,...,NaN,NaN,-0.106090,NaN,0.752188,4.200667e-09,5.209458e-09,3.063959e-09,6.0,NaN
1823,0.066978,-8.036203,0.064442,0.088877,NaN,NaN,NaN,NaN,NaN,1.005693,...,NaN,NaN,-0.106090,NaN,0.752188,4.200667e-09,5.209458e-09,3.063959e-09,NaN,3.0
1824,0.046613,-8.036203,0.064442,0.088877,NaN,NaN,NaN,NaN,NaN,1.041509,...,NaN,NaN,-0.027959,NaN,0.752188,4.369372e-09,5.366691e-09,3.417647e-09,1.0,NaN


In [40]:
train_X, train_Y = to_X_Y(data_21)

In [41]:
xgc_cong = XGBClassifier()
xgc_cong.fit(train_X, train_Y)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [57]:
xgc_cong_alt = XGBClassifier()
xgc_cong_alt.fit(alt_X, alt_Y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [42]:
data_22 = to_intersect("2022Test.csv", ret)
test_data = pd.merge(data_22, df_big, how="left", on=["date", "permno"])
test_alt = pd.merge(data_22, df_big, how="inner", on=["date", "permno"])

/var/folders/sz/ts8tdp9n3f133bj7r6__4w7h0000gn/T/ipykernel_46634/2689625411.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dfRET =  dfRET[df_returns["date"]<=df['yyyymm'].max()]


In [60]:
#test_X, test_Y = to_X_Y(data_22)
t_X, t_Y = to_X_Y(test_alt)

In [46]:
y_pred = xgc_cong.predict(test_X)

In [47]:
metrics.accuracy_score(test_Y, y_pred)
#oof slightly lower than random guessing
#congressional factors perhaps doesn't make a difference

0.3157228825993578

In [62]:
y_pred_alt = xgc_cong_alt.predict(t_X)

In [63]:
metrics.accuracy_score(t_Y, y_pred_alt)

0.24232456140350878

In [72]:
def take_positions_cong(df_big):    
    pos_buy=[]
    pos_sell=[]
    for index, val in df_big.iterrows():
        pos_buy.append(val["buy"])
        pos_sell.append(val["sell"])

        
        
    return pos_buy, pos_sell
#intersection_df_2010.insert(0, "Positions", take_positions(intersection_df_2010, Y_2010_pred), False)

In [73]:
def returns_from_positions(position_data, df_rtn):
    return (position_data * df_rtn).sum(axis=1)#/ (position_data.abs().sum(axis=1))

In [74]:
pos_buy, pos_sell = take_positions_cong(df_big)

In [75]:
len(pos_buy)

10262

In [88]:
def gen_returns(df_big):
    pos_buy, pos_sell = take_positions_cong(df_big)
    #df_big.insert(0, "pos_buy", pos_buy, False)
    #df_big.insert(0, "pos_sell", pos_sell, False)
    pivoted_buy = df_big.pivot_table(index='date', columns='PERMNO', values='pos_buy', aggfunc='mean')
    pivoted_sell = df_big.pivot_table(index="date", columns="PERMNO", values="pos_sell", aggfunc="mean")
    pivoted_ret = df_big.pivot_table(index='date', columns='PERMNO', values='RET', aggfunc='mean')
    return returns_from_positions(pivoted_buy, pivoted_ret), returns_from_positions(pivoted_sell, pivoted_ret)

In [70]:
df_big = pd.merge(df_big, ret, how="left", on=["date", "PERMNO"])

In [89]:
profit_buy, profit_sell = gen_returns(df_big)

In [90]:
profit_buy.sum()
profit_sell.sum()

21.47058499999998

In [91]:
df_big = df_big.rename(columns={"permno":"PERMNO"})

In [ ]:
ret

In [ ]:
df_big

In [ ]:
df_big["buy"] = df_big["buy"].fillna(0)
df_big["sell"] = df_big["sell"].fillna(0)

In [ ]:
df_big

In [ ]:
df_big

In [87]:
df_big["RET"] = pd.to_numeric(df_big["RET"], errors='coerce')

In [ ]:
df_big.dropna(subset=["RET"], inplace=True)

In [92]:
def sharpe_ratio(monthly_return_series):
    return round(monthly_return_series.mean() / monthly_return_series.std() * 12**0.5, 2)

In [93]:
sharpe_ratio(profit_buy)

0.04

In [94]:
sharpe_ratio(profit_sell)

0.11

In [ ]:
df_docs = pd.read_csv("SignalDoc.csv")

In [ ]:
df_docs[df_docs["Acronym"]=="iomom_supp"]

In [69]:
df_big = df_big.rename(columns={"permno": "PERMNO"})

In [81]:
df_big

,pos_sell,pos_buy,date,PERMNO,Type,buy,sell,TICKER,COMNAM,RET
0,NaN,2.0,2021-04-01,10107,buy,2.0,NaN,MSFT,MICROSOFT CORP,0.069602
1,NaN,1.0,2021-04-01,12060,buy,1.0,NaN,GE,GENERAL ELECTRIC CO,-0.000762
2,NaN,3.0,2021-04-01,13169,buy,3.0,NaN,ZNGA,ZYNGA INC,0.059745
3,NaN,1.0,2021-04-01,14505,buy,1.0,NaN,ENLC,ENLINK MIDSTREAM LLC,0.101107
4,NaN,2.0,2021-04-01,14980,buy,2.0,NaN,SHLX,SHELL MIDSTREAM PARTNERS LP,0.156672
...,...,...,...,...,...,...,...,...,...,...
10257,1.0,NaN,2023-12-01,92655,sell,NaN,1.0,UNH,UNITEDHEALTH GROUP INC,-0.044523
10258,1.0,NaN,2023-12-01,93132,sell,NaN,1.0,FTNT,FORTINET INC,0.113584
10259,1.0,NaN,2023-12-01,93246,sell,NaN,1.0,GNRC,GENERAC HOLDINGS INC,0.103955
10260,2.0,NaN,2023-12-01,93427,sell,NaN,2.0,FN,FABRINET,0.175602
